In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
import math

seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid=pd.read_csv("sample 80mm.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#add elapsed time columns
timestamps1=pd.to_datetime(mid["mass 2.0_time"],format="%Y/%m/%d %H:%M:%S")
timestamps2=pd.to_datetime(mid["mass 28.0_time"],format="%Y/%m/%d %H:%M:%S")
timestamps3=pd.to_datetime(mid["mass 44.0_time"],format="%Y/%m/%d %H:%M:%S")

runtime1=(timestamps1-timestamps1[0]).dt.total_seconds()    
mid.insert(1,"Elapsed time, mass 2",runtime1)
runtime2=(timestamps2-timestamps2[0]).dt.total_seconds()    
mid.insert(10,"Elapsed time, mass 28",runtime2)
runtime3=(timestamps3-timestamps2[0]).dt.total_seconds()    
mid.insert(16,"Elapsed time, mass 44",runtime3)
mid.head()


mass 2.0_time  Elapsed time, mass 2  mass 2.0_value  \
0  2022/11/15 15:30:06.754                 0.000    7.148700e-12   
1  2022/11/15 15:30:09.077                 2.323    6.675700e-12   
2  2022/11/15 15:30:14.401                 7.647    6.995500e-12   
3  2022/11/15 15:30:16.724                 9.970    6.943800e-12   
4  2022/11/15 15:30:19.047                12.293    6.888700e-12   

             mass 4.0_time  mass 4.0_value           mass 12.0_time  \
0  2022/11/15 15:30:07.004    1.612000e-13  2022/11/15 15:30:07.254   
1  2022/11/15 15:30:09.327    7.520000e-14  2022/11/15 15:30:09.828   
2  2022/11/15 15:30:14.651    3.294000e-13  2022/11/15 15:30:14.901   
3  2022/11/15 15:30:16.974    3.428000e-13  2022/11/15 15:30:17.224   
4  2022/11/15 15:30:19.297    2.891000e-13  2022/11/15 15:30:19.547   

   mass 12.0_value           mass 16.0_time  mass 16.0_value  \
0     3.762000e-13  2022/11/15 15:30:07.505     4.300000e-13   
1     2.680000e-14  2022/11/15 15:30:10.078     4.300000e-13   
2     1.225000e-13  2022/11/15 15:30:15.151     2.424000e-13   
3     8.400000e-14  2022/11/15 15:30:17.475     2.591000e-13   
4     7.410000e-14  2022/11/15 15:30:19.798     2.511000e-13   

            mass 18.0_time  ...  mass 18.0_value           mass 28.0_time  \
0  2022/11/15 15:30:07.755  ...     6.450000e-13  2022/11/15 15:30:08.005   
1  2022/11/15 15:30:10.328  ...     2.418000e-13  2022/11/15 15:30:10.578   
2  2022/11/15 15:30:15.401  ...     5.729000e-13  2022/11/15 15:30:15.651   
3  2022/11/15 15:30:17.725  ...     5.513000e-13  2022/11/15 15:30:17.975   
4  2022/11/15 15:30:20.048  ...     5.727000e-13  2022/11/15 15:30:20.298   

  mass 28.0_value           mass 32.0_time mass 32.0_value  \
0    5.912000e-13  2022/11/15 15:30:08.255    3.762000e-13   
1    1.070000e-14  2022/11/15 15:30:11.079    1.827000e-13   
2    4.891000e-13  2022/11/15 15:30:15.902    3.440000e-14   
3    4.489000e-13  2022/11/15 15:30:18.225    1.900000e-14   
4    4.481000e-13  2022/11/15 15:30:20.548    3.310000e-14   

   Elapsed time, mass 44           mass 40.0_time mass 40.0_value  \
0                  0.750  2022/11/15 15:30:08.505    3.762000e-13   
1                    NaN                      NaN             NaN   
2                  8.397  2022/11/15 15:30:16.152    3.760000e-14   
3                    NaN  2022/11/15 15:30:18.726    2.182000e-13   
4                    NaN  2022/11/15 15:30:21.049    2.147000e-13   

            mass 44.0_time mass 44.0_value  
0  2022/11/15 15:30:08.755    5.912000e-13  
1                      NaN             NaN  
2  2022/11/15 15:30:16.402    2.273000e-13  
3                      NaN             NaN  
4                      NaN             NaN  

[5 rows x 21 columns]

### Position of the EGA

 

<div>
<img src="20221114_144204.jpg" width="1000"/>
</div>

In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

Live comments                 Time  Elapsed time  injection 100mbar  \
1            NaN  15/11/2022 15:28:48           7.0          -0.063527   
2            NaN  15/11/2022 15:28:56          15.0          -0.063035   
3            NaN  15/11/2022 15:29:03          22.0          -0.063798   
4            NaN  15/11/2022 15:29:10          29.0          -0.064760   
5            NaN  15/11/2022 15:29:17          36.0          -0.062615   

       Barion_2      Barion_1  DUAL experiment  DUAL insulation  \
1  5.850000e-10  3.170000e-10     4.996000e-09     5.254000e-07   
2  5.830000e-10  3.160000e-10     4.996000e-09     5.248000e-07   
3  5.830000e-10  3.160000e-10     4.996000e-09     5.248000e-07   
4  5.840000e-10  3.160000e-10     4.996000e-09     5.248000e-07   
5  5.830000e-10  3.160000e-10     4.996000e-09     5.204000e-07   

   injection 1mbar  helium  T-platinum  T-CERNOX  I_emission  I_grid  
1         0.000485   239.0      -5.163  4302.676         NaN     NaN  
2         0.000484   239.0      -5.275  4302.789         NaN     NaN  
3         0.000486   239.0      -5.163  4302.901         NaN     NaN  
4         0.000478   239.0      -5.163  4302.901         NaN     NaN  
5         0.000498   239.0      -5.275  4302.789         NaN     NaN

In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



Elapsed time        VMon1        VMon2        VMon3        VMon4  \
count   3366.000000  3366.000000  3366.000000  3366.000000  3366.000000   
mean    2018.228343   242.886631    98.063654   978.245051   977.929834   
std     1164.458191   249.417247    13.972747   144.616752   145.801451   
min        1.172000     0.040000     0.060000     0.000000     0.000000   
25%     1010.336000     0.100000   100.060000   999.960000  1000.060000   
50%     2018.473000     0.140000   100.060000   999.980000  1000.060000   
75%     3026.082750   499.920000   100.060000   999.980000  1000.080000   
max     4033.971000   499.920000   100.140000   999.980000  1000.080000   

             VMon5   VMon6   VMon7         VMon8        IMon1        IMon2  \
count  3366.000000  3366.0  3366.0  3.366000e+03  3366.000000  3366.000000   
mean    977.428176     0.0     0.0  2.000000e-02    -0.058453     0.532728   
std     147.317038     0.0     0.0  1.113858e-15     0.085515     0.529528   
min       0.000000     0.0     0.0  2.000000e-02    -0.493900     0.006400   
25%    1000.060000     0.0     0.0  2.000000e-02    -0.139800     0.019200   
50%    1000.060000     0.0     0.0  2.000000e-02    -0.000400     0.022600   
75%    1000.080000     0.0     0.0  2.000000e-02     0.019400     1.027600   
max    1000.080000     0.0     0.0  2.000000e-02     0.534500     3.303100   

             IMon3        IMon4        IMon5        IMon6        IMon7  \
count  3366.000000  3366.000000  3366.000000  3366.000000  3366.000000   
mean     12.071718    12.089546    12.072743    -0.009342    -0.016986   
std       1.812004     1.819521     1.824252     0.001704     0.001228   
min       0.009400     0.008000     0.006200    -0.014400    -0.019400   
25%      12.308000    12.329200    12.314100    -0.010100    -0.017800   
50%      12.444250    12.464100    12.449200    -0.009000    -0.017000   
75%      12.532525    12.552500    12.538500    -0.008100    -0.015900   
max      12.634500    12.656400    12.641500    -0.006500    -0.014400   

             IMon8       hv_grid            em  
count  3366.000000  3.366000e+03  3.366000e+03  
mean     -0.011663  7.947017e-08  5.327284e-07  
std       0.000769  6.643241e-08  5.295278e-07  
min      -0.013600  0.000000e+00  6.400000e-09  
25%      -0.012200  1.940000e-08  1.920000e-08  
50%      -0.011600  2.340000e-08  2.260000e-08  
75%      -0.010900  1.398000e-07  1.027600e-06  
max      -0.010200  5.345000e-07  3.303100e-06

### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
#x=pd.Timestamp(2022, 11, 15, 14,18, 53)

#then find the time value closest to it in the MID scan file - NOT IN USE
#print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck
print("Res 1:",mid["mass 2, corr"].describe())


#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][78:729]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:77]/h2_sens
base_h2.describe()

Result 1:           Date          Time  Elapsed time  VMon1   VMon2   VMon3   VMon4  \
78  15/11/2022  15:31:13.646        95.204   0.06  100.14  999.14  893.14   
79  15/11/2022  15:31:14.843        96.401   0.12  100.14  999.30  982.62   
80  15/11/2022  15:31:16.035        97.593   0.08  100.12  999.46  999.60   
81  15/11/2022  15:31:17.197        98.755   0.08  100.12  999.54  999.60   

     VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
78  680.88    0.0    0.0  ...  1.3459   9.8469   9.7662   9.3660 -0.0144   
79  770.96    0.0    0.0  ...  1.3413  10.7313  10.6603  10.3895 -0.0144   
80  901.12    0.0    0.0  ...  1.0154  10.9129  10.8848  10.8783 -0.0144   
81  992.74    0.0    0.0  ...  1.2089  11.2947  11.2655  11.2742 -0.0144   

     IMon7   IMon8      Comment       hv_grid        em  
78 -0.0182 -0.0109  emission on  9.800000e-09  0.000001  
79 -0.0182 -0.0109  emission on  8.200000e-08  0.000001  
80 -0.0182 -0.0109  emission on  8.200000e-08  0.

count    7.700000e+01
mean     3.182654e-10
std      3.002044e-12
min      3.056639e-10
25%      3.165231e-10
50%      3.183306e-10
75%      3.197625e-10
max      3.278705e-10
Name: mass 2, corr, dtype: float64

### Plotting

In [7]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(mid['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='h2')
plt.plot(mid['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='co')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("RGA scans overlay")
plt.yscale('log')


In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current recording")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 14,
        }
plt.text(150, 1.9e-6, 'Emission on', fontdict = font)
plt.text(1150, 7e-9, 'Grid repelling', fontdict = font)
plt.text(2070, 1.9e-6, 'Grid transparent', fontdict = font)
plt.text(2750, 7e-9, 'Grid repelling, EGA turned', fontdict = font)
plt.text(3570, 4.5e-8, 'Grid transparent', fontdict = font)

plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2 pressure reading')
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1 pressure reading')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Barion 2 pressure evolution")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
ax.yaxis.set_minor_formatter(LogFormatter(minor_thresholds=(2,0.5)))
plt.show()

In [10]:
#Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]/3600),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution")
plt.yscale('linear')